## Introduction

In [38]:
#imports
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc, calinski_harabasz_score, silhouette_score, davies_bouldin_score
import numpy as np

from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV, StratifiedKFold, cross_val_predict
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis, LinearDiscriminantAnalysis
from sklearn.naive_bayes import CategoricalNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import OrdinalEncoder

import pandas as pd
from numpy.typing import NDArray
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

from sklearn.metrics import accuracy_score, f1_score

np.random.seed(1234)

In [39]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

### Metrics 

We have a classification problem with a strong imbalance on the target class. This time we will asume equally important the missclassification errors for both classes. For this reason we are going to use the next metrics to evaluate our model:
* Accuracy. (Just for checking, accuracy is not the best metric with imbalanced data)
* Precision for all classes. 
* Recall for all classes. 
* F1-score micro average.
* F1-score macro average. 


In [40]:
def get_metrics(y_pred: NDArray, y_test: pd.core.series.Series) -> list:
    accuracy = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred, average='weighted')
    precision = precision_score(y_test, y_pred, average='weighted')
    f1_score_w = f1_score(y_test, y_pred, average='weighted')
    return [accuracy, recall, precision, f1_score_w]

results = pd.DataFrame(columns=['Accuracy', 'Recall', 'Precision', 'F1-score weighted'])
results_test = pd.DataFrame(columns=['Accuracy', 'Recall', 'Precision', 'F1-score weighted'])

# MODEL CLASSIFICATION

First of all, read the preprocessed data. It is important to be this data and not the original, because they are saved in a .csv file as it was done in the preprocessing.

In [20]:
# read clean data
X_train = pd.read_csv('../data/X_train.csv')
y_train = pd.read_csv('../data/y_train.csv')['Severity']
X_test = pd.read_csv('../data/X_test.csv')
y_test = pd.read_csv('../data/y_test.csv')['Severity']

### Resampling protocl

It is very important to make a good resampling protocol.

For further information, check out the report. As a resume, we have two datasets, one for training and another one for validation.

# MODELS OF DECISION TREE FAMILY

## Model 1: Decision Tree Classifier

First model we are going to take into account is a Decision Tree Classifier due to its simplicity and interpretability and the fact that it is a good starting point for more complex models. As explained detailedly in the report, we are going to use a cross validation to find the best hyperparameters for this model based on validation metric: f1-score micro.

In [ ]:
criterion = ['gini', 'entropy']

max_dephts = [None, 5, 10, 15]
min_samples_split = [1, 2, 3, 4, 5]
min_samples_leaf = [1, 2, 3, 4, 5]
max_features = ['sqrt', 'log2', None]

dec_tree_model = DecisionTreeClassifier(random_state=1234)

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=1234)

grid_search = GridSearchCV(estimator=dec_tree_model,
                   param_grid={
                          'criterion': criterion,
                          'max_depth': max_dephts,
                          'min_samples_split': min_samples_split,
                          'min_samples_leaf': min_samples_leaf,
                          'max_features': max_features
                   },
                   scoring='f1_weighted',
                   cv=cv,
                   n_jobs=-1
)

grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [None, 5, 10, 15],
                         'max_features': ['sqrt', 'log2', None],
                         'min_samples_leaf': [1, 2, 3, 4, 5],
                         'min_samples_split': [1, 2, 3, 4, 5]},
             scoring='f1_weighted')

Fit the decision tree classifier with best hyperparameters and evaluate it with the test set.

In [ ]:
best_params_dec_tree = grid_search.best_params_
best_dec_tree_model = DecisionTreeClassifier(**best_params_dec_tree, random_state=1234)
best_dec_tree_model.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=15, min_samples_leaf=5, min_samples_split=5)

In [ ]:
y_pred = cross_val_predict(best_dec_tree_model, X_train, y_train, cv=5, n_jobs=-1)

results.loc['Decision Tree'] = get_metrics(y_pred, y_train)
results.sort_values(by='F1-score weighted', ascending=False)

,Accuracy,Recall,Precision,F1-score weighted,F1-score macro.
Decision Tree,0.822593,0.822593,0.800702,0.807829,0.474993


## Model 2: Random Forest Classifier

Second model we are going to take into account is a Random Forest Classifier. This model is an ensemble of decision trees, which makes it more robust to overfitting and more accurate than a single decision tree. As explained detailedly in the report, we are going to use a cross validation to find the best hyperparameters for this model based on validation metric: f1-score weighted.

In [ ]:
rf_model = RandomForestClassifier(random_state=12)

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=1234)

param_grid = {
    'n_estimators': [100, None],
    'max_depth': [3, 5, None],
    'min_samples_split': [2, 3, 5],
    'min_samples_leaf': [1, 2, 3]
}

grid_search = GridSearchCV(rf_model, param_grid = param_grid, cv = cv, scoring='f1_weighted', n_jobs=-1)
grid_search.fit(X_train, y_train)
best_params_rf = grid_search.best_params_

In [ ]:
best_rf_model = RandomForestClassifier(**best_params_rf, random_state=1234)    
best_rf_model.fit(X_train, y_train)

RandomForestClassifier(min_samples_split=3)

In [ ]:
y_pred = cross_val_predict(best_rf_model, X_train, y_train, cv=5, n_jobs=-1)

results.loc['Random Forest'] = get_metrics(y_pred, y_train)
results.sort_values(by='F1-score weighted', ascending=False)

,Accuracy,Recall,Precision,F1-score weighted,F1-score macro.
Decision Tree,0.822593,0.822593,0.800702,0.807829,0.474993
Random Forest,0.836390,0.836390,0.817577,0.805335,0.423240


## Model 3: Extra Trees Classifier

Extra Trees creates multiple highly randomized trees and averages their predictions to reduce variance. It uses random splits and the full dataset for training each tree, making it faster and robust to overfitting compared to Random Forest.

In [ ]:
extra_trees_model = ExtraTreesClassifier(random_state=1234)

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=1234)

param_grid = {
    'n_estimators': [100, None],
    'max_depth': [3, 5, None],
    'min_samples_split': [2, 3, 5],
    'min_samples_leaf': [1, 2, 3]
}

grid_search = GridSearchCV(extra_trees_model, param_grid = param_grid, cv = cv, scoring='f1_weighted', n_jobs=-1)

grid_search.fit(X_train, y_train)
best_params_extra_trees = grid_search.best_params_

In [ ]:
best_extra_trees_model = ExtraTreesClassifier(**best_params_extra_trees, random_state=1234)
best_extra_trees_model.fit(X_train, y_train)

In [ ]:
y_pred = cross_val_predict(best_extra_trees_model, X_train, y_train, cv=5, n_jobs=-1)

results.loc['Extra Trees'] = get_metrics(y_pred, y_train)
results.sort_values(by='F1-score weighted', ascending=False)

## Model 4: LOGISTIC REGRESSION

Search the optimal value of the hyperparameter C and the penalty method for the Logistic Regression model using cross validation. We are going to use the same metric as in the Random Forest Classifier: f1-score weighted.

In [ ]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=1234)
logreg_model = LogisticRegression(random_state = 42)

param_grid = {
    'penalty': ['l1', 'l2'],
    'C': [0.01, 0.1, 1, 10],
    'solver': ['saga']
}

grid_search = GridSearchCV(logreg_model, param_grid = param_grid, cv = cv, scoring='f1_weighted', n_jobs=-1)

grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
best_score = grid_search.best_score_

In [ ]:
bes_logreg_model = LogisticRegression(**best_params, random_state=12)
bes_logreg_model.fit(X_train, y_train)

In [ ]:
y_pred = cross_val_predict(bes_logreg_model, X_train, y_train, cv=5, n_jobs=-1)

results.loc['Logistic Regression'] = get_metrics(y_pred, y_train)
results.sort_values(by='F1-score weighted', ascending=False)

## Model 5: QDA

This model is used to predict the class from an input feature space. The model is based on prior probabilities on the input data. It assumes they are Gaussian distributed, so it was important to normalize the data in the preprocessing.

In [ ]:
#We will do a 5-fold cross-validation to find the best hyperparameters for the QDA model
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=1234)
qda_model = QuadraticDiscriminantAnalysis(random_state = 42)

param_grid = {'reg_param': [0, 0.001, 0.01, 0.1, 0.2, 0.5, 1]}

grid_search = GridSearchCV(qda_model, param_grid = param_grid, cv = cv, scoring='f1_weighted', n_jobs=-1)

grid_search.fit(X_train, y_train)

best_params_qda = grid_search.best_params_
best_score_qda = grid_search.best_score_

In [ ]:
best_qda_model = QuadraticDiscriminantAnalysis(**best_params_qda, random_state=1234)
best_qda_model.fit(X_train, y_train)

In [ ]:
y_pred = cross_val_predict(best_qda_model, X_train, y_train, cv=5, n_jobs=-1)

results.loc['QDA'] = get_metrics(y_pred, y_train)
results.sort_values(by='F1-score weighted', ascending=False)

## Model 6: LDA

This model is very similar to the previous one, here we will suppose that every class in Severity has the same covariance matrix.

In [ ]:
#We will do a 5-fold cross-validation to find the best hyperparameters for the LDA model
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=1234, )

param_grid = {'solver': ['lsqr', 'eigen'], 'shrinkage': [0, 0.1, 0.2, 0.5]}
lda_model = LinearDiscriminantAnalysis()
grid_search = GridSearchCV(lda_model, param_grid = param_grid, cv = cv, scoring='f1_weighted', n_jobs=-1)

grid_search.fit(X_train, y_train)

best_params_lda = grid_search.best_params_
best_score_lda = grid_search.best_score_

In [ ]:
best_lda_model = LinearDiscriminantAnalysis(**best_params_lda, random_state=1234)
best_lda_model.fit(X_train, y_train)

In [ ]:
y_pred = cross_val_predict(best_lda_model, X_train, y_train, cv=5, n_jobs=-1)

results.loc['LDA'] = get_metrics(y_pred, y_train)
results.sort_values(by='F1-score weighted', ascending=False)

The results for QDA and LDA are not the same, it may be due to te fact that every class does not have the same variance-covariance matrix, that is the assumption made for doing LDA.

## Model 7: GAUSSIAN NAIVE BAYE

Gaussian Naive Bayes (GNB) is a simple probabilistic classifier based on Bayes' theorem with the assumption of independence between features. In the preprocessing we saw that there is not a lot of correlation between variables, so we can apply this method.

In [21]:
#We will do a 5-fold cross-validation to find the best hyperparameters for the GNB model
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=1234)

#For the priors we will use the default values of None and the proportion of every class in training the dataset
class_counts = y_train.value_counts()
class_counts_sorted = class_counts.sort_index()
class_proportions = class_counts_sorted / len(y_train)
param_grid = {'class_prior': [None, class_proportions.values], 'fit_prior': [True, False], 'alpha': [0.1,0.5,1]}

gnb_model = CategoricalNB(random_state = 12)

grid_search = GridSearchCV(gnb_model, param_grid = param_grid, cv = cv, scoring='f1_weighted', n_jobs=-1)

#Before fitting the model, we have to encode our training data in order to get good results
X_encoded = OrdinalEncoder().fit_transform(X_train)
grid_search.fit(X_encoded, y_train)
best_params_gnb = grid_search.best_params_
best_score_gnb = grid_search.best_score_

In [22]:
best_gnb_model = CategoricalNB(**best_params_gnb, random_state = 1234)
best_gnb_model.fit(X_encoded, y_train)

CategoricalNB(alpha=0.1)

In [24]:
y_pred = cross_val_predict(best_gnb_model, X_encoded, y_train, cv=5, n_jobs=-1)

results.loc['GNB'] = get_metrics(y_pred, y_train)
results.sort_values(by='F1-score weighted', ascending=False)

,Accuracy,Recall,Precision,F1-score weighted
GNB,0.705553,0.705553,0.716562,0.710759


As we see this models does not fit well the data. This could be due to we supposed variables were uncorrelated, but in fact they are. Thus there exists dependece between variables, and this method cannot be applied.

## Model 8: $k$-NN

This methods predicts the actual sample by using similar properties from the k nearest neighbours. Each new sample will go to the class with the most similar values from the train set. 

In [ ]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=1234 )

param_grid = {'n_neighbors': [1, 3, 10, 20], 'metric': ['euclidean', 'minkowski', 'manhattan']}
knn_model = KNeighborsClassifier(random_state = 12)
grid_search = GridSearchCV(knn_model, param_grid = param_grid, cv = cv, scoring='f1_weighted', n_jobs=-1)
grid_search.fit(X_train, y_train)

best_params_knn = grid_search.best_params_

In [ ]:
best_knn_model = KNeighborsClassifier(**best_params_knn, random_state=1234)
best_knn_model.fit(X_train, y_train)

In [ ]:
y_pred = cross_val_predict(best_knn_model, X_train, y_train, cv=5, n_jobs=-1)

results.loc['KNN'] = get_metrics(y_pred, y_train)
results.sort_values(by='F1-score weighted', ascending=False)

## Model 9: Clustering

Finally, we will apply clustering with 4 clusters, the same number of classes at Severity. It is a great method to agroup data. We will aply two methods, k-Means and EM.

### Model 9.1: $k$-Means

In [26]:
#We will do a 5-fold cross-validation to find the best hyperparameters for the LDA model
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=1234)

param_grid = {'n_clusters': [4], 'init':['kmeans++','random']}
kmeans_model = KMeans()
grid_search = GridSearchCV(kmeans_model, param_grid = param_grid, cv = cv, scoring='f1_weighted', n_jobs=-1)
grid_search.fit(X_train, y_train)
best_params_kmeans = grid_search.best_params_

In [28]:
best_kmeans_model = KMeans(**best_params_kmeans)
best_kmeans_model.fit(X_train, y_train)

KMeans(init='random', n_clusters=4)

In [37]:
y_pred_kmeans = best_kmeans_model.predict(X_test)

CH = calinski_harabasz_score(X_test, y_pred_kmeans)
SS = silhouette_score(X_test, y_pred_kmeans)
DB = davies_bouldin_score(X_test, y_pred_kmeans)

print(CH, SS, DB)

79811.37245234697 0.2925299427387994 1.281567933246841


### Model 9.2: EM

In [32]:
#We will do a 5-fold cross-validation to find the best hyperparameters for the LDA model
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=1234, )

param_grid = {'n_components': [4], 'init_params':['kmeans','random']}
em_model = GaussianMixture()
grid_search = GridSearchCV(em_model, param_grid = param_grid, cv = cv, scoring='f1_weighted', n_jobs=-1)
grid_search.fit(X_train, y_train)
best_params_em = grid_search.best_params_

In [33]:
best_em_model = GaussianMixture(**best_params_em)
best_em_model.fit(X_train, y_train)

GaussianMixture(init_params='random', n_components=4)

In [36]:
y_pred_em = best_em_model.predict(X_test)

CH = calinski_harabasz_score(X_test, y_pred_em)
SS = silhouette_score(X_test, y_pred_em)
DB = davies_bouldin_score(X_test, y_pred_em)

results.loc['KMeans'] = [CH, SS, DB]

ValueError: Expected 2D array, got 1D array instead:
array=[2 2 2 ... 3 2 3].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

## Testing best models with the test set

### Decision Tree Classifier

In [ ]:
# best_dec_tree_model
y_pred = best_dec_tree_model.predict(X_test)
results_test.loc['Decision Tree'] = get_metrics(y_pred, y_test)
results_test.sort_values(by='F1-score weighted', ascending=False)

In [ ]:
fpr = dict()
tpr = dict()
roc_auc = dict()

# Get the predicted probabilities for each class
y_pred_proba = best_dec_tree_model.predict_proba(X_test)

# Compute ROC curve and ROC AUC for each class
for i in range(4):
    fpr[i], tpr[i], _ = roc_curve(y_test == i+1, y_pred_proba[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Plot ROC curve for each class
plt.figure()
colors = ['blue', 'red', 'green', 'orange']
for i in range(4):
    plt.plot(fpr[i], tpr[i], color=colors[i], label='Severity {} (AUC = {:.2f})'.format(i+1, roc_auc[i]))
plt.plot([0, 1], [0, 1], color='black', linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.show()

In [ ]:
feature_importances = best_dec_tree_model.feature_importances_

# Create a dataframe to store the feature importances
importance_df = pd.DataFrame({'Feature': X_train.columns, 'Importance': feature_importances})

# Sort the dataframe by importance in descending order
importance_df = importance_df.sort_values(by='Importance', ascending=False)

# Plot the feature importances
plt.figure(figsize=(10, 6))
plt.bar(importance_df['Feature'], importance_df['Importance'])
plt.xticks(rotation=90)
plt.xlabel('Feature')
plt.ylabel('Importance')
plt.title('Feature Importances')
plt.show()

### Random Forest Classifier

In [ ]:
y_pred = best_rf_model.predict(X_test)
results_test.loc['Random Forest'] = get_metrics(y_pred, y_test)
results_test.sort_values(by='F1-score weighted', ascending=False)

In [ ]:
fpr = dict()
tpr = dict()
roc_auc = dict()

# Get the predicted probabilities for each class
y_pred_proba = best_rf_model.predict_proba(X_test)

# Compute ROC curve and ROC AUC for each class
for i in range(4):
    fpr[i], tpr[i], _ = roc_curve(y_test == i+1, y_pred_proba[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Plot ROC curve for each class
plt.figure()
colors = ['blue', 'red', 'green', 'orange']
for i in range(4):
    plt.plot(fpr[i], tpr[i], color=colors[i], label='Severity {} (AUC = {:.2f})'.format(i+1, roc_auc[i]))
plt.plot([0, 1], [0, 1], color='black', linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.show()


In [ ]:
feature_importances = best_rf_model.feature_importances_

# Create a dataframe to store the feature importances
importance_df = pd.DataFrame({'Feature': X_train.columns, 'Importance': feature_importances})

# Sort the dataframe by importance in descending order
importance_df = importance_df.sort_values(by='Importance', ascending=False)

# Plot the feature importances
plt.figure(figsize=(10, 6))
plt.bar(importance_df['Feature'], importance_df['Importance'])
plt.xticks(rotation=90)
plt.xlabel('Feature')
plt.ylabel('Importance')
plt.title('Feature Importances')
plt.show()